# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [ ]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.8 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [ ]:
# your code here
nlen = 2048

def generate_keys():
  p = cu.getStrongPrime(nlen // 2)
  q = cu.getStrongPrime(nlen // 2)
  # 3, 4, 5
  while p <= pow(2, (nlen - 1) // 2) or q <= pow(2, (nlen - 1) // 2) or abs(p - q) <= pow(2, nlen / 2 - 100):
      p = cu.getStrongPrime(nlen // 2)
      q = cu.getStrongPrime(nlen // 2)
      
  n = p * q
  f = (p - 1) * (q - 1)
  
  # 1, 2
  e = cu.getRandomRange(65537, min(f, pow(2, 256)))
  while cu.GCD(f, e) != 1 or math.lcm(p - 1, q - 1) < e * pow(2, nlen // 2):
        e = cu.getRandomRange(65537, min(f, pow(2, 256)))  
  
  d = pow(e, -1, f)
  
  return (e, d, n)


## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [ ]:
M = "Ala ma kota"
# Your code here

e, d, n = generate_keys()

C = pow(int.from_bytes(M.encode('utf-8'), 'big'), e, n)
M_encoded = C.to_bytes(math.ceil(C.bit_length() / 8), 'big')

print(M_encoded.hex())


t = pow(int.from_bytes(M_encoded, 'big'), d, n)
M_decoded = t.to_bytes(math.ceil(t.bit_length() / 8), 'big').decode('utf-8')

print(M_decoded)


6ecac16d0f5740fb911445b77c86e6f67e5d338f3f29e9ff8e758b9aa0bc2be233a8c4540498dff363ec1d71e7a441fedf73857e19cbe65964fa9f3966f3994b0a9056fd59b7fafbf99a1f98328b2998560c7b97cb4970ed8681a3b70ac7bd7284867178676f787effc23704b7b2045d9e3f24f947f1ad61031816998462646f0562c89e91817df8a5fd21a23a0df3ee62217449a201f384ef28bcfe2b0495ebe29e34b5adbd67c6606ad86cbf3503b08669e1a40ff013ae7bd77a45047d32a59976598b918e424e7dbd4599ff4c3bcf64f2e7c64ec65edc33787c8ea0c91f472711c388db8f2c572047a1022569f1b37a4832292107c54f3379148a3ccf3405
Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [ ]:
# Your code here
from hashlib import sha256

def create_sign(m, d, n):
  sha = sha256()
  sha.update(m)
  h = cu.bytes_to_long(sha.digest())
  sig = pow(h, d, n)
  return m, sig

def verify_sign(m, sig, e, n):
  sha = sha256()
  sha.update(m)
  h = cu.bytes_to_long(sha.digest())
  ver = pow(sig, e, n)
  return ver == h

m, sig = create_sign(M.encode('utf-8'), d, n)
print(verify_sign(m, sig, e, n))
print(verify_sign(m + b'01', sig, e, n))

True
False


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [ ]:
# Your code here
hlen = 32
def MGF1(m, size, h=sha256):
  output = bytearray()
  for i in range(0, math.ceil(size / hlen)):
    i_bytes = i.to_bytes(4, 'big')
    tmp = sha256(m + i_bytes).digest()
    output += tmp
  return output[: size]


test = MGF1(bytes.fromhex("00123212"), 20)
print(test.hex())
print(len(test))

2eb85d5173c8dc56a0f0e3b20e42695f2bd71a68
20


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [87]:
M = "Ala ma kota"

# Your code here
from random import randbytes

def OAEP_encode(k, mlen, m, h=sha256, mgf1=MGF1):
  if mlen > k - 2 * hlen - 2:
    raise Exception("Error")

  l_hash = h(b"").digest()
  PS = bytearray(k - mlen - 2 * hlen - 2)
  DB = l_hash + PS + b'\x01' + m
  print("len: ", len(DB), k - hlen - 1)
  seed = randbytes(hlen)
  db_mask = mgf1(seed, k - hlen - 1)
  masked_DB = bytearray([DB[i] ^ db_mask[i] for i in range(len(DB))])
  seed_mask = mgf1(masked_DB, hlen)
  masked_seed = bytearray([seed[i] ^ seed_mask[i] for i in range(len(seed))])
  EM = b'\x00' + masked_seed + masked_DB
  print("len: ", len(EM), k)
  return EM


def OAEP_decode(k, mlen, EM, h=sha256, mgf1=MGF1):
  y = EM[0]
  masked_seed = EM[1 : hlen + 1]
  masked_DB = EM[hlen + 1 :]
  l_hash = h(b"").digest()
  seed_mask = mgf1(masked_DB, hlen)
  seed = bytearray([masked_seed[i] ^ seed_mask[i] for i in range(len(masked_seed))])
  db_mask = mgf1(seed, k - hlen - 1)
  DB = bytearray([masked_DB[i] ^ db_mask[i] for i in range(len(masked_DB))])
  
  l_hash2 = DB[:hlen]
  DB = DB[hlen:]

  i = 0
  while DB[i] != 1: 
      i += 1
  
  if DB[i] == 1: 
      i += 1
  
  m = DB[i:]
  
  if y != 0 or l_hash != l_hash2:
    raise Exception("Error")

  return m


def RSA_OAEP_encoding(M, mlen, nlen):
  M = OAEP_encode(nlen, mlen, M)
  M_encoded = int.from_bytes(M, 'big')
  C = pow(M_encoded, e, n)
  return C

def RSA_OAEP_decoding(C, mlen, nlen):
  M_decoded = pow(C, d, n).to_bytes(nlen, 'big')
  M = OAEP_decode(nlen, mlen, M_decoded)
  return M


M_bytes = M.encode('utf-8')
print(M_bytes.hex())
print(OAEP_encode(100, len(M_bytes), M_bytes).hex())
C = RSA_OAEP_encoding(M_bytes, len(M_bytes), 100)
M = RSA_OAEP_decoding(C, len(M_bytes), 100).decode('utf-8')
print(M)

416c61206d61206b6f7461
len:  67 67
len:  100 100
00a25d9387681b5a2e9a00e1e4b92dfc8f2450a812a43265d8a1f2a65e5a881a9986483af628e2cb168043e1151adc645a60e53f65eeca092dc76e87cc8186d23b8b2c6efdaea45a4e8d51f5f562fdb1a0183721e41129bfab56a85ef31ca826f2ac4d66
len:  67 67
len:  100 100
Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [85]:
m = "Ala ma podpis"

# Your code here

def EMSA_encoding(M, emlen, h=sha256, em_bits=2047, slen=32, mgf1=MGF1):
  M_hash = h(M).digest()

  if emlen < hlen + slen + 2:
      raise Exception("Error")

  salt = randbytes(slen)
  M_p =b'\x00' * 8 + M_hash + salt
  print(len(M_p), 8 + hlen + slen)
  M_hash_p = h(M_p).digest()
  PS = bytearray(emlen - slen - hlen - 2)
  DB = PS + b'\x01' + salt
  DB_mask = mgf1(M_hash_p, emlen - hlen - 1)
  masked_DB = bytearray([DB[i] ^ DB_mask[i] for i in range(len(DB))])
  masked_DB[:8 * emlen - em_bits] = b'\x00' * (8 * emlen - em_bits)
  return masked_DB + M_hash_p + b'\xbc'


def EMSA_decoding(EM, M, emlen, h=sha256, em_bits=2047, slen=32, mgf1=MGF1):
  M_hash = h(M).digest()

  if emlen < hlen + slen + 2:
    raise Exception("Error")
  if EM[-1] != 188:
    raise Exception("Error")

  masked_DB = EM[: emlen - hlen - 1]
  EM = EM[emlen - hlen - 1:]
  
  m_hash2 = EM[:hlen]
  EM = EM[hlen:]
  
  if masked_DB[:8 * emlen - em_bits] != b'\x00' * (8 * emlen - em_bits):
    raise Exception("Error")

  DB_mask = mgf1(m_hash2, emlen - hlen - 1)
  DB = bytearray([masked_DB[i] ^ DB_mask[i] for i in range(len(masked_DB))])
  DB[: 8 * emlen - em_bits] = b'\x00' * (8 * emlen - em_bits)

  if DB[:emlen - hlen - slen - 2] != b'\x00' * (emlen - hlen - slen - 2) or DB[emlen - hlen - slen - 2] != 1:
    raise Exception("Error")

  salt = DB[-slen:]
  m2 = b'\x00' * 8 + M_hash + salt
  print(len(m2), 8 + hlen + slen)
  m_hash3 = h(m2).digest()

  if(m_hash2 != m_hash3):
    raise Exception("Error")
  print("OK")


m1 = "Bob ma podpis"

EM = EMSA_encoding(m.encode('utf-8'), 100)
EMSA_decoding(EM, m.encode('utf-8'), 100)

EM = EMSA_encoding(m1.encode('utf-8'), 256)
EMSA_decoding(EM, m1.encode('utf-8'), 256)

72 72
72 72
OK
72 72
72 72
OK


In [98]:
def EMSA_sign(m, d, n, l):
  encoded = EMSA_encoding(m, l)
  h = int.from_bytes(encoded, 'big')
  sig = pow(h, d, n)
  return m, sig

def EMSA_verify(m, sig, e, n, l):
  ver = pow(sig, e, n)
  return EMSA_decoding(ver.to_bytes(ver.bit_length() / 8, 'big'), m, l)

m, sig = EMSA_sign(M.encode('utf-8'), d, n, 100)
print(EMSA_verify(m, sig, e, n, 100))

72 72
72 72
OK
None


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [103]:
# Your code here
def encrypt_KEM(l):
  rand = randbytes(320)
  key = sha256(rand).digest()
  ck = RSA_OAEP_encoding(key, 32, l)
  return key, ck

def decrypt_KEM(ck, l):
  key = RSA_OAEP_decoding(ck, 32, l)
  return key

key, ck = encrypt_KEM(256)
print(key.hex(), ck)

key2 = decrypt_KEM(ck, 256)
print(key2.hex())

len:  223 223
len:  256 256
499485a4372d02401bca62a96dae4c29cfc1c78751e4054a9fcec253ebfceae3 22656369248149533598373079262121693902270021244415166995335487695332145035347131142135575208517109305065807164951482984954128300851253540500264538865910425531535464346503678583545301183582821968797064669093487453000828886069413914188600639845608546048838731536252867045054668863933747649241669088520836085130111179216359392724030445913673627367001994415557683624262034412960258113875926620303436136011709012449918896818228434134340129266403453456378742887732537251252306277938043445686774918402991750261164828507516531585416230539180913589924876194504303109629195889350322182685038148486828153640937487376809320797845
499485a4372d02401bca62a96dae4c29cfc1c78751e4054a9fcec253ebfceae3
